In [13]:
import os
from dotenv import load_dotenv
import streamlit as st

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_qdrant import Qdrant
from tavily import TavilyClient

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from datetime import datetime

In [14]:
# --- Load API keys from .env ---
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [15]:
# --- LLM Setup ---
primary_qa_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

In [ ]:
# -----------------------------
# 1. CACHE HEAVY OBJECTS
# -----------------------------
@st.cache_resource
def load_llm():
    return ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

@st.cache_resource
def load_vectorstore():
    loader = PyMuPDFLoader("data/2025-DOST-SEI-ST-Scholars-Handbook-Final.pdf")
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600,
        chunk_overlap=150,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    chunks = text_splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

    return Qdrant.from_documents(
        chunks,
        embeddings,
        location=":memory:",
        collection_name="DOST_Handbook",
    )

In [ ]:
# --- Init core retrievers ---
llm = load_llm()
qdrant_vector_store = load_vectorstore()

retriever = qdrant_vector_store.as_retriever(search_kwargs={"k": 7})
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
)
advanced_retriever = MultiQueryRetriever.from_llm(
    retriever=compression_retriever,
    llm=llm,
)

2025-09-26 12:07:19.718 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 12:07:19.940 
  command:

    streamlit run c:\Users\Mark Gelson\Desktop\Predictive Systems AI\Demo Day\day_4\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-09-26 12:07:19.941 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 12:07:19.941 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 12:07:19.941 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 12:07:19.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 12:07:19.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [20]:
# --- Custom Prompt (encourages detailed answers) ---
from langchain.prompts import ChatPromptTemplate

handbook_prompt = PromptTemplate.from_template( """
You are an assistant for DOST-SEI scholars ans wering questions using the official handbook.

CRITICAL INSTRUCTIONS:
1. Answer ONLY based on the provided context below
2. Do NOT add information not present in the context
3. If you're unsure, say "I need more specific information from the handbook" or "Can you give mo more context to your question?"
4. Quote specific sections when possible

Context:
{context}

Question:
{input}

Answer clearly and concisely.
""" )

handbook_chain = LLMChain(llm=primary_qa_llm, prompt=handbook_prompt)

C:\Users\Mark Gelson\AppData\Local\Temp\ipykernel_38200\1455303568.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  handbook_chain = LLMChain(llm=primary_qa_llm, prompt=handbook_prompt)


In [21]:
# --- Document Chain and Retrieval Chain ---

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import LLMChain, create_retrieval_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, handbook_prompt)
retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [22]:
# --- Guardrails ---
def guardrails_pre_query(user_input: str) -> str:
    blocked = ["violence", "politics", "religion"]
    if any(b in user_input.lower() for b in blocked):
        return "Sorry, I cannot answer that type of question."
    return user_input

def guardrails_post_response(response: str) -> str:
    if "I don't know" in response or len(response.strip()) < 20:
        response += "\n\n⚠️ This answer may be incomplete. Please verify in the official handbook."
    return response

In [23]:
# --- Judge Chain ---
fallback_prompt = PromptTemplate.from_template("""
You are a judge
Given the user question and the handbook answer,
decide if the answer is sufficient, accurate, and complete.

Question: {question}
Answer: {answer}

Respond with ONLY one word: "sufficient" or "insufficient".
""")
judge_chain = LLMChain(llm=primary_qa_llm, prompt=fallback_prompt)

In [24]:
from datetime import datetime
from tavily import TavilyClient

tavily = TavilyClient(api_key=TAVILY_API_KEY)

def tavily_search(query: str, max_results: int = 5) -> tuple[str, list]:
    # Step 1: Try trusted domains
    preferred_domains = [
        "sei.dost.gov.ph",
        "dost.gov.ph",
        "facebook.com/DOST.SEI",
        "facebook.com/DOSTph",
        "science-scholarships.ph",
    ]

    results = tavily.search(
        query,
        max_results=max_results,
        include_domains=preferred_domains
    )

    # Step 2: Fall back to open web if nothing
    if not results["results"]:
         results = tavily.search(query, max_results=max_results)

    # Step 3: Format with dates
    parsed = []
    summaries = []
    for r in results["results"]:
        pub_date = r.get("published_date")
        if pub_date:
            try:
                pub_date = datetime.fromisoformat(pub_date.replace("Z", "+00:00"))
                date_str = pub_date.strftime("%Y-%m-%d")
            except Exception:
                date_str = "N/A"
        else:
            date_str = "N/A"

        summaries.append(f"[{date_str}] {r['title']}: {r['content']}")
        parsed.append({"title": r["title"], "url": r["url"], "date": date_str})

    return "\n".join(summaries), parsed


In [28]:
# --- Stronger web prompt that requires recency & citations ---
web_prompt = PromptTemplate.from_template("""
You are an assistant for DOST-SEI scholars. Use ONLY the top search results below to answer the question.
IMPORTANT:
- PRIORITIZE the most recent information.
- If the results do not contain recent/definitive information, reply exactly: "I couldn't find recent information on this topic."

Top search results (most recent first):
{results}

Question:
{question}

Answer (use only the search results above):
""")

web_chain = LLMChain(llm=primary_qa_llm, prompt=web_prompt)

In [29]:
@st.cache_data(show_spinner="Answering ...")
def hybrid_agent(question: str, history: list = None) -> str:
    """
    Hybrid agent with handbook RAG + web fallback.
    Accepts optional conversation history for follow-ups.
    Uses Streamlit caching to speed up repeated queries.
    """

    # --- Step 0: Use conversation memory ---
    context_from_history = ""
    if history:
        # Take only last 3 exchanges to keep context focused
        context_from_history = "\n\nRecent conversation:\n" + "\n".join(
            [f"{m['role']}: {m['content']}" for m in history[-3:]]
        )

    # Append history to user question
    full_query = question + context_from_history

    # --- Step 1: Guardrails (pre-query) ---
    safe_query = guardrails_pre_query(full_query)
    if safe_query.startswith("Sorry"):
        return safe_query

    # --- Step 2: Handbook RAG ---
    handbook_response = retrieval_chain.invoke({"input": safe_query})
    answer = handbook_response["answer"]
    answer = guardrails_post_response(answer)

    # --- Step 3: Judge sufficiency ---
    judge_result = judge_chain.invoke({"question": safe_query, "answer": answer})
    if "insufficient" in judge_result["text"].strip().lower():
        results_text, parsed_results = tavily_search(safe_query)
        web_answer = web_chain.invoke({
            "results": results_text,
            "question": safe_query
        })
        return web_answer["text"]

    return answer


2025-09-26 12:17:09.804 No runtime found, using MemoryCacheStorageManager
